<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/notebooks/023_Tokenization_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧩 Tokenizacja: Dlaczego AI nie umie literować?

Dla człowieka zdanie *"Kocham AI"* to dwa słowa i 8 liter (plus spacja).
Dla komputera to ciąg liczb.

Mamy trzy podejścia do cięcia tekstu:
1.  **Char-level:** Każda litera to osobna liczba (a=1, b=2).
    *   *Wada:* Teksty są strasznie długie. Model gubi wątek.
2.  **Word-level:** Każde słowo to liczba (dom=500, kot=200).
    *   *Wada:* Co ze słowami "domku", "domkiem", "przydomowy"? Słownik robi się gigantyczny.
3.  **Sub-word (BPE - Byte Pair Encoding):** Złoty środek.
    *   Częste słowa ("the", "and", "jest") są jednym kawałkiem.
    *   Rzadkie słowa ("przydomowy") są sklejane z kawałków ("przy", "dom", "owy").

To standard w dzisiejszym AI (GPT, Llama, Claude).

In [2]:
# Najpierw musimy zainstalować bibliotekę od OpenAI
# Odkomentuj linię poniżej, jeśli uruchamiasz to pierwszy raz:
!uv pip install tiktoken

import tiktoken

# Ładujemy "słownik" którego używa GPT-4
# cl100k_base to nazwa kodowania dla GPT-3.5 i GPT-4
encoding = tiktoken.get_encoding("cl100k_base")

print("Enkoder załadowany. Zobaczmy, jak GPT-4 widzi świat.")

Enkoder załadowany. Zobaczmy, jak GPT-4 widzi świat.


Using Python 3.13.2 environment at: venv
Resolved 7 packages in 772ms
Prepared 2 packages in 942ms
Installed 2 packages in 13ms
 + regex==2025.11.3
 + tiktoken==0.12.0


## Eksperyment 1: Słowo vs Token

Sprawdźmy proste zdanie. Zobaczysz, że czasem token to całe słowo (wraz ze spacją!), a czasem kawałek słowa.

In [3]:
text = "Sztuczna inteligencja jest super"

# 1. Zamieniamy tekst na liczby (Tokeny)
tokens = encoding.encode(text)

print(f"Tekst oryginalny: '{text}'")
print(f"Tokeny (ID): {tokens}")
print(f"Liczba słów: {len(text.split())}")
print(f"Liczba tokenów: {len(tokens)}")

print("-" * 30)
print("JAK GPT WIDZI TE KAWAŁKI?")

# 2. Zamieniamy liczby z powrotem na tekst, kawałek po kawałku
for t in tokens:
    decoded = encoding.decode([t])
    print(f"ID {t:<6} -> '{decoded}'")

Tekst oryginalny: 'Sztuczna inteligencja jest super'
Tokeny (ID): [50, 11949, 98667, 3458, 90745, 967, 5697, 13599, 2307]
Liczba słów: 4
Liczba tokenów: 9
------------------------------
JAK GPT WIDZI TE KAWAŁKI?
ID 50     -> 'S'
ID 11949  -> 'zt'
ID 98667  -> 'ucz'
ID 3458   -> 'na'
ID 90745  -> ' intelig'
ID 967    -> 'enc'
ID 5697   -> 'ja'
ID 13599  -> ' jest'
ID 2307   -> ' super'


## Eksperyment 2: Problem "Strawberry" (i matematyki)

Dlaczego ChatGPT myli się w prostych zadaniach?
Spójrzmy na słowo "Strawberry" i działanie matematyczne.

Jeśli model widzi token "berry" jako jedną całość (jedną liczbę), to **nie ma fizycznej możliwości**, żeby wiedział, ile tam jest liter "r", chyba że nauczył się tego na pamięć. On nie widzi liter w środku tokena!

In [4]:
tricky_words = ["Strawberry", "9.11", "9.9"]

print("--- ANALIZA TRUDNYCH SŁÓW ---")

for word in tricky_words:
    ids = encoding.encode(word)
    parts = [encoding.decode([id]) for id in ids]
    
    print(f"Słowo: '{word}'")
    print(f"Tokeny: {parts}")
    print(f"ID:     {ids}")
    print("-" * 20)

# ANALIZA WYNIKÓW (Dla Twojej informacji):
# 1. "Strawberry" -> Często dzieli się na ['Str', 'aw', 'berry'].
#    Model widzi 3 klocki. W klocku 'berry' są dwa 'r', ale model widzi to jako jeden symbol.
# 2. "9.11" vs "9.9" ->
#    9.9 to często jeden token.
#    9.11 to czasem dwa tokeny ['9.', '11'].
#    Dlatego modele słabo liczą na liczbach zmiennoprzecinkowych - traktują je jak tekst!

--- ANALIZA TRUDNYCH SŁÓW ---
Słowo: 'Strawberry'
Tokeny: ['Str', 'aw', 'berry']
ID:     [2645, 675, 15717]
--------------------
Słowo: '9.11'
Tokeny: ['9', '.', '11']
ID:     [24, 13, 806]
--------------------
Słowo: '9.9'
Tokeny: ['9', '.', '9']
ID:     [24, 13, 24]
--------------------


## Eksperyment 3: Język ma znaczenie (Angielski vs Polski)

Tokenizatory są trenowane głównie na angielskim internecie.
To sprawia, że angielski jest "tańszy" (mniej tokenów na zdanie) i model lepiej go rozumie.
Polski jest "droższy" (słowa są szatkowane na więcej drobnych kawałków, bo mają końcówki fleksyjne).

**Płacisz za tokeny, nie za słowa!**

In [5]:
text_en = "I verify that the calculations are correct."
text_pl = "Potwierdzam, że obliczenia są poprawne."

tokens_en = encoding.encode(text_en)
tokens_pl = encoding.encode(text_pl)

print(f"Angielski: '{text_en}'")
print(f"Liczba tokenów: {len(tokens_en)}")
print(f"Tokeny: {[encoding.decode([t]) for t in tokens_en]}")
print("\n")

print(f"Polski: '{text_pl}'")
print(f"Liczba tokenów: {len(tokens_pl)}")
print(f"Tokeny: {[encoding.decode([t]) for t in tokens_pl]}")

diff = (len(tokens_pl) - len(tokens_en)) / len(tokens_en) * 100
print("-" * 30)
print(f"Wniosek: To samo zdanie w języku polskim zajmuje {diff:.0f}% więcej tokenów (pamięci i pieniędzy)!")

Angielski: 'I verify that the calculations are correct.'
Liczba tokenów: 8
Tokeny: ['I', ' verify', ' that', ' the', ' calculations', ' are', ' correct', '.']


Polski: 'Potwierdzam, że obliczenia są poprawne.'
Liczba tokenów: 15
Tokeny: ['Pot', 'w', 'ier', 'dz', 'am', ',', ' że', ' ob', 'lic', 'zenia', ' są', ' pop', 'raw', 'ne', '.']
------------------------------
Wniosek: To samo zdanie w języku polskim zajmuje 88% więcej tokenów (pamięci i pieniędzy)!


## 🧠 Podsumowanie: Świat z Klocków Lego

Co musisz zapamiętać jako AI Engineer?

1.  **Token $\neq$ Słowo.** Średnio 1000 słów to około 1300 tokenów (w angielskim). W polskim ta różnica jest jeszcze większa.
2.  **Ślepota modelu.** Model widzi tokeny jako niepodzielne klocki. Jeśli token `12345` oznacza "Strawberry", to model nie wie, z jakich liter się składa, dopóki nie zobaczy tokenów literowych `S-t-r-a-w...`.
3.  **Optymalizacja kosztów.** Pisząc prompty, pamiętaj, że język polski zjada limit kontekstu szybciej niż angielski. Czasami warto przetłumaczyć instrukcję systemową na angielski, nawet jeśli bot ma odpowiadać po polsku.

**Tu jest haczyk.**
Dlatego GPT-4 jest lepsze w kodowaniu niż w pisaniu wierszy rymowanych. Kod ma jasną strukturę tokenów. Rymy wymagają słyszenia brzmienia końcówek słów, a tokenizator często te końcówki ucina w dziwnych miejscach.